> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# Python SQL
Python oferece um ecossistema robusto para gerência de banco de dados, com suporte aos mais diferentes bancos de dados, como MySQL, PostgreSQL, Oracle, entre outros. Nessa aula, daremos os primeiros passos conhecendo o básico do Python para trabalhar com SQL. Especificamente falaremos das seguintes bibliotecas:
* [**sqlite3**](https://docs.python.org/3/library/sqlite3.html): Interface para bancos de dados SQLite nativa do Python. Alternativa leve e autocontida, onde os dados são armazenados em um único arquivo e não requer configuração de servidor. **Usa-se a linguagem SQL** para interações com o banco.
* [**MySQL Connector**](https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html): A biblioteca `mysql-connector` é um driver de comunicação com servidores MySQL que podemos instalar através do gerenciador de pacotes `pip`. Ela permite se conectar a servidores MySQL, criar tabelas, executar consultas, entre outros, **usando a linguagem SQL**.
* [**✨SQLAlchemy** 🧙](https://www.sqlalchemy.org/): Um toolkit completo e robusto que oferece tanto a modelagem de dados (ORM - Object-Relational Mapping) quanto a gerência do banco de dados. SQLAlchemy abstrai as interações com o banco de dados, permitindo **trabalhar com objetos Python em vez de consultas SQL brutas.**

### Base de dados exemplo
Os dados (e suas respectivas modelagens) que usamos nesse script foram ligeiramente modificados desse dataset do Kaggle: [Pizza Query](https://www.kaggle.com/datasets/teocalvo/pizzaquery/data?select=produto.csv). A origem dos dados é o curso SQL Essentials da [Linux Tips](https://www.linuxtips.io/) em parceria com [Téo Me Why](https://github.com/teomewhy).

A seguir uma representação simples das tabelas que iremos trabalhar.

```
produto
+------------+------+
| idProduto  | PK   |
| tipo       |      |
| descItem   |      |
| vlPreco    |      |
+------------+------+

pedido
+------------+------+
| idPedido   | PK   |
| dtPedido   |      |
| flKetchup  |      |
| descUF     |      |
| txtRecado  |      |
+------------+------+

item_pedido
+------------+-------------------+
| idPedido   | PK, FK -> pedido  |
| idProduto  | PK, FK -> produto |
| quantidade |                   |
+------------+-------------------+
```




## SQLite

### "Conectando" ao database

Por não se tratar de um banco cliente-servidor, a conexão ao banco é na verdade o acesso ao arquivo (ou criação dele) que contém as informações do banco. A conexão pode ser feita com o seguinte código:

```python
import sqlite3
connection = sqlite3.connect(path)
```
O parâmetro `path` em geral é o nome do arquivo onde os registros estão/estarão armazenados. Mas também pode ser a string `:memory:` caso se deseje criar um banco provisório em memória, o qual deixará de existir quando a conexão for fechada.

O código a seguir apresenta uma função mais robusta para abrir uma nova conexão SQLite, incluindo o tratamento de erros mapeado pelo seguinte instrumento:
* [`sqlite3.Error`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Error): Classe base para todas as exceções dessa biblioteca (herda da python `Exception`). Exceções lançadas pela própria biblioteca tem dois atributos:
    * `sqlite_errorcode`: Código de erro da [API SQLite](https://sqlite.org/rescode.html).
    * `sqlite_errorname`: Nome simbólico do erro.

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("pizza_app.sqlite")

Connection to SQLite DB successful


### Executando queries

O objeto retornado `connection`, retornado pela conexão que acabamos de criar, possui um método `cursor()` que cria e retorna um novo objeto `Cursor`. O objeto `Cursor` é um intermediário para enviar comandos SQL para o banco de dados e recuperar resultados (veja a [documentação](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor)). Principais métodos:

* `execute`: Executa uma instrução SQL, recebida como string. Opcionalmente pode-se adicionar placeholders à string e preenchê-los em `parameters`. [Entenda como adicionar placeholders em queries SQL](https://docs.python.org/3/library/sqlite3.html#how-to-use-placeholders-to-bind-values-in-sql-queries).
```python
execute(sql: str, parameters: tuple = ())
# exemplo com placeholders nomeados:
dic = ({'ing': 'queijo', 'preco': 4.5})
cur.execute("INSERT INTO data VALUES(:ing, :preco)", dic)
```
* `executemany`: Repete a execução da instrução `sql` dada com cada um dos elementos de `parameters`.
```python
executemany(sql: str, parameters: Sequence[tuple])
# exemplo com ? placeholders
rows = [("queijo", 4.5), ("azeitona", 1)]
cur.executemany("INSERT INTO data VALUES(?, ?)", rows)
```
* `executescript`: Permite executar scripts sql.
```python
executescript(sql_script: str)
#exemplo
cur.executescript("""
    BEGIN;
    CREATE TABLE person(firstname, lastname, age);
    CREATE TABLE book(title, author, published);
    COMMIT;
""")
```

> **`rowcount`**: Atributo da classe `Cursor` com o número de linhas afetadas por instruções `INSERT`, `UPDATE`, `DELETE` ou `REPLACE`. É atualizado pelos méetodos `execute()` e `executemany()`. Se nenhuma linha foi afetada pela operação, é `-1`.

> ❗ Instrução `INSERT` ❗ abre uma [transação](https://docs.python.org/3/library/sqlite3.html#sqlite3-controlling-transactions) que fica pendente até que se invoque `connection.commit()`. Só então as atualizações são salvas no banco de dados. Caso nenhuma transação esteja pendente, o commit não faz nada.


In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)

        # commit necessário para INSERT
        connection.commit() ###

        print(f"Query executada.")
        if cursor.rowcount != -1:
            print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")

In [ ]:
# Cria a tabela produto #
create_produto_table = \
"""CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
);"""

execute_query(connection, create_produto_table)
#########################

# Cria a tabela pedido #
create_pedido_table = \
"""CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
);"""

execute_query(connection, create_pedido_table)
#########################

# Cria a tabela item_pedido #
create_item_pedido_table = \
"""CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);"""
execute_query(connection, create_item_pedido_table)
#########################

Query executada.
Query executada.
Query executada.


In [ ]:
## Inserindo registros manualmente

# Inserindo produto #
insert_produto = \
"""INSERT INTO
produto (tipo, desc_item, vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', 9.25),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection, insert_produto)
######################

# Inserindo pedido
insert_pedido = \
"""INSERT INTO
pedido (dt_pedido, fl_ketchup, desc_uf, txt_recado)
VALUES
('2023-06-01', TRUE, 'MG', 'Capricha no queijo!');
"""
execute_query(connection, insert_pedido)
######################

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas


In [ ]:
# Inserindo item_pedido
itens = (
    {'id_pedido': 1, 'id_produto': 2, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 3, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 1, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 4, 'qtd': 2},
    {'id_pedido': 1, 'id_produto': 5, 'qtd': 3}
)

insert_item_pedido = \
"""INSERT INTO item_pedido (id_pedido, id_produto, quantidade)
VALUES(:id_pedido, :id_produto, :qtd);"""

cursor = connection.cursor()
cursor.executemany(insert_item_pedido, itens)
connection.commit()
cursor.close()

### Executando queries de leitura

Após executar uma instrução de consulta ao banco através do objeto `Cursor`, o resultado da consulta pode ser recuperado invocando um dos seguintes métodos do cursor:

* `fetchone`: Retorna a próxima linha do resultado da consulta. Cada nova chamada a fetchone() retorna a linha subsequente (como um cursor avançando após ler uma linha). Seu retorno é uma única tupla com os dados do registro.
```python
fetchone() -> tuple
```
* `fetchall`: Retorna uma lista de tuplas com todas as linhas resultantes da consulta. Cada tupla é um registro do banco.
```python
fetchall() -> list[tuple]
```
* `fetchmany`: Retorna o próximo conjunto de linhas da consulta, de acordo com parâmetro `size`. Se `size` não for especificado, o valor padrão é `cursor.arraysize`. Também retorna uma lista de tuplas.
```python
fetchmany(size: int = cur.arraysize) -> list[tuple]
```

In [ ]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

A primeira consulta que faremos será a verificação das tabelas que criamos. Todo dados SQLite tem uma "tabela de esquema" que armazena o esquema desse banco de dados. A tabela é representada a seguir e você pode ver mais detalhes sobre ela na [documentação da API](https://www.sqlite.org/schematab.html).
```
CREATE TABLE sqlite_schema(
  type text,
  name text,
  tbl_name text,
  rootpage integer,
  sql text
);
```

Vamos criar uma query para ler o nome de todas as tabelas registradas no esquema:
```sql
SELECT name FROM sqlite_master WHERE type='table';
```

Note no resultado que além das tabelas que nós criamos, também existe uma tabela extra:
* [`sqlite_sequence`](https://www.sqlite.org/fileformat2.html#seqtab): É uma tabela interna para implementar `AUTOINCREMENT`. É criada automaticamente sempre que qualquer tabela comum com uma chave primária inteira AUTOINCREMENT é criada. Uma vez criada, existirá na tabela `sqlite_schema` para sempre; não pode ser descartada.

In [ ]:
select_table_names = \
"SELECT name FROM sqlite_master WHERE type='table';"
tables = execute_read_query(connection, select_table_names)
print(tables, '\n')

for table in tables:
    select_all = f"SELECT * FROM {table[0]}"
    res = execute_read_query(connection, select_all)
    print(f"{table[0]}: {res}")

[('produto',), ('sqlite_sequence',), ('pedido',), ('item_pedido',)] 

produto: [(1, 'ingrediente', 'camarão', 6), (2, 'massa', 'tradicional', 9.25), (3, 'borda', 'tradicional', 0), (4, 'queijo', 'muçarela', 4), (5, 'bebida', 'refrigerante', 5)]
sqlite_sequence: [('produto', 5), ('pedido', 1)]
pedido: [(1, '2023-06-01', 1, 'MG', 'Capricha no queijo!')]
item_pedido: [(1, 2, 1), (1, 3, 1), (1, 1, 1), (1, 4, 2), (1, 5, 3)]


### Dropando...

A seguir vamos dropar todos os registros de todas as tabelas que criamos manualmente, para a seguir conhecer uma maneira mais interessante de carregamento de dados. Não façam isso em ~casa~ projetos reais 😆

In [ ]:
execute_query(connection, "DELETE FROM item_pedido;")
execute_query(connection, "DELETE FROM pedido;")
execute_query(connection, "DELETE FROM produto;")

Query executada.
5 linha(s) afetadas
Query executada.
1 linha(s) afetadas
Query executada.
5 linha(s) afetadas


### Carregando arquivos CSV 🐼

Antes de tudo, baixe a versão modificada dos dados do Pizza Query.

In [ ]:
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/produto.csv

--2024-09-05 20:10:15--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103557 (101K) [text/plain]
Saving to: ‘item_pedido.csv’

item_pedido.csv     100%[===================>] 101.13K  --.-KB/s    in 0.02s   

2024-09-05 20:10:16 (4.08 MB/s) - ‘item_pedido.csv’ saved [103557/103557]

--2024-09-05 20:10:16--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting

Para carregar tabelas em bancos sqlite, o Pandas entra em cena com o método `.to_sql()`. Ele facilita a exportação de DataFrames para bancos de dados SQL [(veja a documentação)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html). Esse método é compatível tanto com conexões do sqlite3 quanto os bancos suportados pelo SQLAlchemy. A seguir está um exemplo de chamada com os principais parâmetros.

```python
df.to_sql(table_name, connection, if_exists='fail', index=False)
```

O parâmetro `if_exists` controla o comportamento caso a tabela já exista no banco de dados, podendo assumir os seguintes valores:

* `'fail'`: Gera um erro se a tabela já existir (padrão).
* `'replace'`: Remove a tabela existente antes de inserir os dados.
* `'append'`: Adiciona os dados ao final da tabela existente.

In [ ]:
import pandas as pd

df_pedido = pd.read_csv(f'pedido.csv')
df_pedido.to_sql('pedido', connection, if_exists='append', index=False)

count_rows = "SELECT COUNT(id_pedido) as count_id FROM pedido;"
print(execute_read_query(connection, count_rows))

select_all = f"SELECT * FROM pedido WHERE id_pedido < 5;"
execute_read_query(connection, select_all)

[(1106,)]


[(0, '2023-05-11', None, 'GO', None),
 (1, '2023-05-11', None, 'PR', 'Aquela pizza perfeita! :-D'),
 (2, '2023-05-11', None, 'SP', 'Muito obrigado!!'),
 (3, '2023-05-11', None, 'SP', None),
 (4, '2023-05-11', None, 'RS', 'Capricha no peperoni')]

Mas cuidado com coisas automáticas! Ao criar tabelas com o `to_sql` (por exemplo definindo `if_exists='replace'`) os tipos de dados serão inferidos automaticamente e podem não ser tipos desejáveis para as regras do banco.

Preste atenção na tabela Pedido que criamos manualmente. Em seguida descomente a linha da célula a seguir e execute novamente. Note que os tipos de dados escolhidos tendem a ser mais genéricos, já que não existe uma compreensão de modelagem do banco.

In [ ]:
df_pedido.to_sql('pedido', connection, if_exists='replace', index=False)
res = execute_read_query(connection, "SELECT sql FROM sqlite_schema")
for r in res:
    print(r[0])

CREATE TABLE produto (
    idProduto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    descItem VARCHAR(100),
    vlPreco DECIMAL(10, 2)
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE pedido (
    idPedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dtPedido DATE,
    flKetchup BOOLEAN,
    descUF CHAR(2),
    txtRecado TEXT
)
CREATE TABLE item_pedido (
    idPedido INT NOT NULL,
    idProduto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (idPedido, idProduto),
    FOREIGN KEY (idPedido) REFERENCES pedido(idPedido),
    FOREIGN KEY (idProduto) REFERENCES produto(idProduto)
)
None


O Pandas é uma interface de comunicação com bancos de dados SQL não só na criação e/ou preenchimento de tabelas, como também permitindo executar queries. A vantagem nesse caso é que o retorno da query é estruturado como um objeto tipo DataFrame que pode ser trabalhando posteriormente no Python

In [ ]:
query="""
SELECT desc_uf, COUNT(*) as count_pedidos
FROM pedido
GROUP BY desc_uf
ORDER BY count_pedidos DESC
LIMIT 5
"""
pd.read_sql_query(query, connection)

,descUF,count_pedidos
0,SP,395
1,RJ,103
2,MG,92
3,PR,76
4,RS,50


## MySQL

O MySQL é um sistema de gerenciamento de banco de dados cliente-servidor. Para seguir esta seção do notebook, é preciso ter um servidor MySQL ativo para se conectar. Abaixo estão instruções resumidas de como configurar um servidor MySQL local:

#### No windows
* [Baixe](https://dev.mysql.com/downloads/installer/) e instale o MySQL Server. Durante a instalação, você vai configurar uma senha para o usuário root. **Lembre-se dessa senha,** pois ela será necessária para acessar o servidor MySQL.
* Inicie o MySQL Server. Ele geralmente inicia automaticamente após a instalação. Caso contrário, procure e inicie o serviço MySQL no painel de serviços do Windows.

#### Sistemas Unix (Mac ou Linux)
* Instalação: Durante a instalação, você vai configurar uma senha para o usuário root. **Lembre-se dessa senha,** pois ela será necessária para acessar o servidor MySQL.
    * No Mac você pode usar o gerenciador de pacotes brew. No terminal execute:
    ```
    brew install mysql
    ```
    * No Linux (distribuições Debian) use o gerenciador de pacotes `apt-get`. No terminal execute:
    ```
    sudo apt-get install mysql-server
    ```
* Inicie o servidor MySQL no terminal:
```
sudo service mysql start
```

> ❗Atenção❗: Execute esse script em uma instância local do Jupyter para acessar o servidor local do MySQL.

### Instale o `mysql-connector-python`
* Developer guide: https://dev.mysql.com/doc/connector-python/en/

In [ ]:
! pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 44.4 MB/s eta 0:00:00


### Criando (e conectando ao) database

Dividimos a criação e conexão ao banco de dados em três passos:
* Conectar ao servidor MySQL sem especificar um banco de dados. Isso nos permite executar comandos a nível de servidor, como a criação de um novo banco de dados.
* Usar a conexão para criar o novo banco de dados. Utilizamos uma função `execute_query` similar a que vimos com o módulo sqlite3.
* Realizar um novo passo de conexão, agora especificando o banco de dados recém-criado. Isso nos permite executar operações diretamente no contexto do nosso banco.

In [ ]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password, db_name=None):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Conexão bem sucedida")
    except Error as e:
        print(f"Erro: '{e}'")

    return connection

# Passo 1: Conectar ao servidor MySQL sem especificar um banco de dados
connection = create_connection("localhost", "root", "12345678")

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()

        print(f"Query executada.")
        if cursor.rowcount != -1:
            print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}' ")

# Passo 2: Criar o banco de dados
create_database_query = "CREATE DATABASE pizza_app"
execute_query(connection, create_database_query)

# Passo 3: Conectar novamente ao servidor, agora especificando o banco de dados
connection = create_connection("localhost", "root", "12345678", "pizza_app")

### Criando tabelas
Com exceção de algumas variações de sintaxe (`AUTOINCREMENT` vs `AUTO_INCREMENT`, chaves estrangeiras nomeadas, etc.), a criação de tabelas é muito similar ao que fizemos com sqlite3.

In [ ]:
# Cria a tabela produto #
create_produto_table = \
"""CREATE TABLE produto (
    id_produto INT AUTO_INCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2),
    PRIMARY KEY (id_produto)
);"""

execute_query(connection, create_produto_table)
#########################

# Cria a tabela pedido #
create_pedido_table = \
"""CREATE TABLE pedido (
    id_pedido INT AUTO_INCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT,
    PRIMARY KEY (id_pedido)
);"""

execute_query(connection, create_pedido_table)
#########################

# Cria a tabela item_pedido #
create_item_pedido_table = \
"""CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY fk_id_pedido  (id_pedido)  REFERENCES pedido(id_pedido),
    FOREIGN KEY fk_id_produto (id_produto) REFERENCES produto(id_produto)
);"""

execute_query(connection, create_item_pedido_table)
#########################

In [ ]:
# Inserindo produto #
insert_produto = \
"""INSERT INTO
produto (tipo, desc_item, vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', 9.25),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection, insert_produto)
######################

### Executando queries de leitura

In [ ]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"Erro: '{e}'")

In [ ]:
tables = execute_read_query(connection, "SHOW TABLES;")
print(tables, '\n')

for table in tables:
    select_all = f"SELECT * FROM {table[0]}"
    res = execute_read_query(connection, select_all)
    print(f"{table[0]}: {res}")

### Dropando...

In [ ]:
execute_query(connection, "DELETE FROM item_pedido;")
execute_query(connection, "DELETE FROM pedido;")
execute_query(connection, "DELETE FROM produto;")